Hello everyone,Recently I participated in a machine learning competition on Hackerearth. https://www.hackerearth.com/challenge/hiring/data-science-hiring-challenge/ .
Data can be found here https://he-s3.s3.amazonaws.com/media/hackathon/data-science-hiring-challenge/funding-successful-projects-c40d55cb-92e5b384/a339bbf8fa8511e7.zip
In this challenge, you have to predict if a project will get successfully funded or not.

I am sharing this so that it helps others in starting with machine learning.

Some basic imports

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

Columns and their descripition:<br>

Variable	            Description<br>

project_id	           - unique id of project<br>
name	               - name of the project<br>
desc	               - description of project<br>
goal	               - the goal (amount) required for the project<br>
keywords	           - keywords which describe project<br>
disable communication  - whether the project authors has disabled communication option with people donating to the                          project<br>
country	               - country of project author<br>
currency	           - currency in which goal (amount) is required<br>
deadline	           - till this date the goal must be achieved (in unix timeformat)<br>
state_changed_at	   - at this time the project status changed. Status could be successful, failed, suspended,                            cancelled etc. (in unix timeformat)<br>
created_at	           - at this time the project was posted on the website(in unix timeformat)<br>
launched_at	           - at this time the project went live on the website(in unix timeformat)<br>
backers_count	       - no. of people who backed the project<br>
final_status	       - whether the project got successfully funded (target variable – 1,0)<br>

In [2]:
#read the train data to  a pandas Dataframe
train=pd.read_csv("./input/train.csv",index_col='project_id')
print (train.shape)
#read the test data to  a pandas Dataframe
actual_test=pd.read_csv('./input/test.csv',index_col='project_id')
print (actual_test.shape)

(108129, 13)
(63465, 11)


as we can see test has 2 columns missing.
1st would be prediction variable but there is another missing colummn.

In [3]:
print ("Train columns")
print (train.columns)
print ("Testset columns")
print (actual_test.columns)

Train columns
Index(['name', 'desc', 'goal', 'keywords', 'disable_communication', 'country',
       'currency', 'deadline', 'state_changed_at', 'created_at', 'launched_at',
       'backers_count', 'final_status'],
      dtype='object')
Testset columns
Index(['name', 'desc', 'goal', 'keywords', 'disable_communication', 'country',
       'currency', 'deadline', 'state_changed_at', 'created_at',
       'launched_at'],
      dtype='object')


As "Backers_count" is missing, I decided not to consider it while training.

In [4]:
# read fist 10 entries of train set to get some idea about data.
train.drop('backers_count',axis=1,inplace=True)
train.head(10)

,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,final_status
project_id,,,,,,,,,,,,
kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,1
kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,0
kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0
kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,1243555740,1243556121,1240963795,1240966730,1
kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1243769880,1243770317,1241177914,1241180541,0
kkst1085176748,daily digest,I'm a fledgling videoblogger living in Brookly...,700.0,daily-digest,False,US,USD,1243815600,1243816219,1241050799,1241464468,0
kkst1468954715,iGoozex - Free iPhone app,I am an independent iPhone developer that beli...,250.0,igoozex-free-iphone-app,False,US,USD,1243872000,1243872028,1241725172,1241736308,0
kkst194050612,Drive A Faster Car 2.0,Drive A Faster Car (http://www.driveafastercar...,1000.0,drive-a-faster-car-20,False,US,USD,1244088000,1244088022,1241460541,1241470291,1
kkst708883590,"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...","Opening Friday, June 5 2009, on view through S...",5000.0,lostles-at-tinys-giant,False,US,USD,1244264400,1244264422,1241415164,1241480901,0


In [5]:
train.info()
#the data types for all columns are also correct

<class 'pandas.core.frame.DataFrame'>
Index: 108129 entries, kkst1451568084 to kkst169301870
Data columns (total 12 columns):
name                     108128 non-null object
desc                     108121 non-null object
goal                     108129 non-null float64
keywords                 108129 non-null object
disable_communication    108129 non-null bool
country                  108129 non-null object
currency                 108129 non-null object
deadline                 108129 non-null int64
state_changed_at         108129 non-null int64
created_at               108129 non-null int64
launched_at              108129 non-null int64
final_status             108129 non-null int64
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 10.0+ MB


In [8]:
#after looking at NA counts less than 10 entries have missing values so dropped them
print(train.count())
train=train.dropna()
print ("\nAfter removing rows with missing values:-")
print(train.count())

name                     108121
desc                     108121
goal                     108121
keywords                 108121
disable_communication    108121
country                  108121
currency                 108121
deadline                 108121
state_changed_at         108121
created_at               108121
launched_at              108121
final_status             108121
dtype: int64

After removing rows with missing values
name                     108121
desc                     108121
goal                     108121
keywords                 108121
disable_communication    108121
country                  108121
currency                 108121
deadline                 108121
state_changed_at         108121
created_at               108121
launched_at              108121
final_status             108121
dtype: int64
